## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-11-21-52-57 +0000,nyt,Brazil’s Supreme Court Is On Course to Convict...,https://www.nytimes.com/2025/09/11/world/ameri...
1,2025-09-11-21-51-00 +0000,wsj,"Microsoft, OpenAI Announce Deal to Extend Part...",https://www.wsj.com/tech/microsoft-openai-anno...
2,2025-09-11-21-50-00 +0000,wsj,Senate Republicans invoked the so-called nucle...,https://www.wsj.com/politics/policy/senate-rep...
3,2025-09-11-21-46-49 +0000,missionlocal,YIMBY rally gets testy ahead of vote on upzoni...,https://missionlocal.org/2025/09/sf-upzoning-r...
4,2025-09-11-21-45-51 +0000,nypost,‘Beautiful’ LI library with secret history – a...,https://nypost.com/2025/09/11/us-news/li-libra...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
54,kirk,71
53,charlie,65
24,trump,37
464,shooting,16
557,epstein,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-09-11-21-39-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...,239
313,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...,227
29,2025-09-11-21-16-12 +0000,nypost,Joe Rogan shocked after learning of Charlie Ki...,https://nypost.com/2025/09/11/us-news/joe-roga...,224
257,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...,220
217,2025-09-11-12-11-58 +0000,latimes,Charlie Kirk shooting reignites campus free-sp...,https://www.latimes.com/california/story/2025-...,202


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,239,2025-09-11-21-39-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...
321,75,2025-09-10-23-40-09 +0000,nypost,UK ambassador Peter Mandelson told Jeffrey Eps...,https://nypost.com/2025/09/10/us-news/uk-ambas...
257,57,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...
88,51,2025-09-11-18-52-58 +0000,nypost,Iryna Zarutska’s boyfriend shares his heartbre...,https://nypost.com/2025/09/11/us-news/iryna-za...
198,48,2025-09-11-13-50-47 +0000,latimes,FBI releases photos of 'person of interest' in...,https://www.latimes.com/world-nation/story/202...
49,43,2025-09-11-20-18-21 +0000,bbc,Bolsonaro's coup trial gripped Brazil - and hi...,https://www.bbc.com/news/articles/cj3yekj2xygo...
63,42,2025-09-11-19-47-56 +0000,nypost,"Desmond Holly, 16, ID’d as Evergreen High Scho...",https://nypost.com/2025/09/11/us-news/desmond-...
239,41,2025-09-11-10-00-00 +0000,latimes,California legislators strike last-minute deal...,https://www.latimes.com/california/story/2025-...
308,40,2025-09-11-00-22-00 +0000,wsj,Senate Republicans voted down a measure to rel...,https://www.wsj.com/politics/policy/senate-rep...
25,36,2025-09-11-21-19-00 +0000,wsj,Paramount Skydance Prepares Ellison-Backed Bid...,https://www.wsj.com/business/media/paramount-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
